# Motor Mode: Start and No-Load Run

This notebook runs an induction motor in motor mode:
- startup from standstill,
- no-load steady-state run.

Focus: detailed stator current analysis for phases **A/B** and resultant magnitude **|I1|**,
with clear visibility of startup peak and steady-state values.


In [ ]:
from pathlib import Path
import sys

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MaxNLocator

cwd = Path.cwd().resolve()
project_root = cwd.parent if cwd.name.lower() == "notebooks" else cwd
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["savefig.facecolor"] = "white"

output_dir = project_root / "output" / "notebooks_currents"
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Figures will be saved to: {output_dir}")


def rms(x: np.ndarray) -> float:
    return float(np.sqrt(np.mean(np.square(x))))


def y_limits(y: np.ndarray, pad: float = 0.08) -> tuple[float, float]:
    y = np.asarray(y)
    ymin = float(np.min(y))
    ymax = float(np.max(y))
    span = max(ymax - ymin, 1e-9)
    return ymin - pad * span, ymax + pad * span


def style_axis(
    ax,
    title: str,
    ylabel: str = "Current, A",
    xbins: int = 18,
    ybins: int = 14,
    x_minor: int = 5,
    y_minor: int = 5,
):
    ax.set_title(title, fontsize=12, weight="bold")
    ax.set_xlabel("Time, s")
    ax.set_ylabel(ylabel)
    ax.ticklabel_format(style="plain", axis="both")
    ax.tick_params(axis="both", labelsize=9)

    ax.xaxis.set_major_locator(MaxNLocator(nbins=xbins, min_n_ticks=8))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=ybins, min_n_ticks=8))
    ax.xaxis.set_minor_locator(AutoMinorLocator(x_minor))
    ax.yaxis.set_minor_locator(AutoMinorLocator(y_minor))

    ax.grid(True, which="major", linewidth=0.9, alpha=0.85, color="#a8a8a8")
    ax.grid(True, which="minor", linestyle=":", linewidth=0.65, alpha=0.75, color="#bdbdbd")


def mark_point_with_projection(
    ax,
    x: float,
    y: float,
    label: str,
    color: str,
    text_offset: tuple[int, int] = (10, 10),
):
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    dx = max(xmax - xmin, 1e-12)
    dy = max(ymax - ymin, 1e-12)

    ax.scatter([x], [y], color=color, s=40, zorder=8)
    ax.vlines(x, ymin, y, colors=color, linestyles="--", linewidth=1.0, alpha=0.95)
    ax.hlines(y, xmin, x, colors=color, linestyles="--", linewidth=1.0, alpha=0.95)

    # Keep projection values inside axes to avoid layout drift.
    ax.text(
        x,
        ymin + 0.03 * dy,
        f"t={x:.4f} s",
        rotation=90,
        va="bottom",
        ha="center",
        color=color,
        fontsize=8,
        clip_on=True,
        bbox=dict(facecolor="white", edgecolor=color, alpha=0.85, pad=1.0),
    )
    ax.text(
        xmin + 0.01 * dx,
        y,
        f"I={y:.1f} A",
        va="center",
        ha="left",
        color=color,
        fontsize=8,
        clip_on=True,
        bbox=dict(facecolor="white", edgecolor=color, alpha=0.85, pad=1.0),
    )

    ax.annotate(
        f"{label}\n(t={x:.4f} s, I={y:.1f} A)",
        xy=(x, y),
        xytext=text_offset,
        textcoords="offset points",
        fontsize=8,
        color=color,
        bbox=dict(facecolor="white", edgecolor=color, alpha=0.9, boxstyle="round,pad=0.25"),
        arrowprops=dict(arrowstyle="->", color=color, lw=1.0),
    )

from core.parameters import MachineParameters
from solvers import ScipySolver, SolverConfig
from scenarios import MotorNoLoadScenario
from simulation import SimulationBuilder


In [ ]:
params = MachineParameters()

solver_cfg = SolverConfig(
    dt_out=5e-5,
    max_step=5e-5,
    rtol=1e-6,
    atol=1e-8,
)

scenario = MotorNoLoadScenario(
    t_end=4.0,
    Mc_idle=0.0,
)

res = (
    SimulationBuilder(params)
    .solver(ScipySolver("RK45", config=solver_cfg))
    .scenario(scenario)
    .run()
)


In [ ]:
t = res.t
iA = res.i1A
iB = res.i1B
I1 = res.I1_mod

startup_end = 0.35
steady_window = 0.40
steady_zoom_len = 0.12

mask_start = t <= startup_end
mask_steady = t >= (t[-1] - steady_window)
mask_steady_zoom = t >= (t[-1] - steady_zoom_len)

t_start = t[mask_start]
iA_start = iA[mask_start]
iB_start = iB[mask_start]
I1_start = I1[mask_start]

t_steady = t[mask_steady]
iA_steady = iA[mask_steady]
iB_steady = iB[mask_steady]
I1_steady = I1[mask_steady]

t_steady_zoom = t[mask_steady_zoom]
iA_steady_zoom = iA[mask_steady_zoom]
iB_steady_zoom = iB[mask_steady_zoom]
I1_steady_zoom = I1[mask_steady_zoom]

# Startup peaks (A/B)
peak_idx_A = int(np.argmax(np.abs(iA_start)))
peak_idx_B = int(np.argmax(np.abs(iB_start)))
t_peak_A = float(t_start[peak_idx_A])
t_peak_B = float(t_start[peak_idx_B])
i_peak_A = float(iA_start[peak_idx_A])
i_peak_B = float(iB_start[peak_idx_B])

# Steady peaks (A/B)
steady_idx_A = int(np.argmax(np.abs(iA_steady)))
steady_idx_B = int(np.argmax(np.abs(iB_steady)))
t_steady_peak_A = float(t_steady[steady_idx_A])
t_steady_peak_B = float(t_steady[steady_idx_B])
i_steady_peak_A = float(iA_steady[steady_idx_A])
i_steady_peak_B = float(iB_steady[steady_idx_B])

# |I1| notable points
I1_peak_start = float(np.max(I1_start))
I1_peak_start_idx = int(np.argmax(I1_start))
t_I1_peak_start = float(t_start[I1_peak_start_idx])

I1_mean_steady = float(np.mean(I1_steady))
t_I1_mean_steady = float(t[-1] - 0.04)

I1_peak_steady_idx = int(np.argmax(I1_steady_zoom))
t_I1_peak_steady = float(t_steady_zoom[I1_peak_steady_idx])
I1_peak_steady = float(I1_steady_zoom[I1_peak_steady_idx])

iA_rms_steady = rms(iA_steady)
iB_rms_steady = rms(iB_steady)

print(f"Peak startup |i1A|: {abs(i_peak_A):.2f} A at t={t_peak_A:.4f} s")
print(f"Peak startup |i1B|: {abs(i_peak_B):.2f} A at t={t_peak_B:.4f} s")
print(f"Peak startup |I1|: {I1_peak_start:.2f} A at t={t_I1_peak_start:.4f} s")
print(f"Steady RMS i1A: {iA_rms_steady:.2f} A")
print(f"Steady RMS i1B: {iB_rms_steady:.2f} A")
print(f"Steady mean |I1|: {I1_mean_steady:.2f} A")


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(18, 15), dpi=220)

# 1) Full time: phase currents A/B
axes[0].plot(t, iA, color="#1f77b4", linewidth=1.0, label="i1A")
axes[0].plot(t, iB, color="#ff7f0e", linewidth=1.0, label="i1B")
axes[0].axvspan(0.0, startup_end, color="#fde2e4", alpha=0.35, label="startup window")
axes[0].axvspan(t[-1] - steady_window, t[-1], color="#e2f0cb", alpha=0.35, label="steady window")
axes[0].set_xlim(t[0], t[-1])
axes[0].set_ylim(*y_limits(np.r_[iA, iB], pad=0.10))
style_axis(axes[0], "Phase Currents A/B - Full Interval", xbins=24, ybins=16, x_minor=5, y_minor=5)
mark_point_with_projection(axes[0], t_peak_A, i_peak_A, "A startup peak", "#1f77b4", (12, 10))
mark_point_with_projection(axes[0], t_peak_B, i_peak_B, "B startup peak", "#ff7f0e", (12, -28))
axes[0].legend(loc="upper right", ncol=2)

# 2) Startup zoom
axes[1].plot(t_start, iA_start, color="#1f77b4", linewidth=1.2, label="i1A")
axes[1].plot(t_start, iB_start, color="#ff7f0e", linewidth=1.2, label="i1B")
axes[1].set_xlim(0.0, startup_end)
axes[1].set_ylim(*y_limits(np.r_[iA_start, iB_start], pad=0.10))
style_axis(axes[1], "Phase Currents A/B - Startup Zoom", xbins=22, ybins=16, x_minor=5, y_minor=5)
mark_point_with_projection(axes[1], t_peak_A, i_peak_A, "A startup peak", "#1f77b4", (12, 10))
mark_point_with_projection(axes[1], t_peak_B, i_peak_B, "B startup peak", "#ff7f0e", (12, -28))
axes[1].legend(loc="upper right")

# 3) Steady-state zoom
axes[2].plot(t_steady_zoom, iA_steady_zoom, color="#1f77b4", linewidth=1.2, label="i1A")
axes[2].plot(t_steady_zoom, iB_steady_zoom, color="#ff7f0e", linewidth=1.2, label="i1B")
axes[2].set_xlim(t_steady_zoom[0], t_steady_zoom[-1])
axes[2].set_ylim(*y_limits(np.r_[iA_steady_zoom, iB_steady_zoom], pad=0.10))
style_axis(axes[2], "Phase Currents A/B - Steady-State Zoom", xbins=18, ybins=16, x_minor=5, y_minor=5)
mark_point_with_projection(axes[2], t_steady_peak_A, i_steady_peak_A, "A steady peak", "#1f77b4", (12, 10))
mark_point_with_projection(axes[2], t_steady_peak_B, i_steady_peak_B, "B steady peak", "#ff7f0e", (12, -28))
axes[2].text(
    0.01,
    0.98,
    (
        f"Steady RMS i1A = {iA_rms_steady:.1f} A\n"
        f"Steady RMS i1B = {iB_rms_steady:.1f} A"
    ),
    transform=axes[2].transAxes,
    va="top",
    ha="left",
    bbox=dict(facecolor="white", alpha=0.9, edgecolor="#999999"),
)
axes[2].legend(loc="upper right")

fig.suptitle("Motor No-Load: Detailed Phase Currents (High Resolution)", fontsize=15, weight="bold")
fig.subplots_adjust(left=0.10, right=0.98, top=0.94, bottom=0.06, hspace=0.36)
phase_save_path = output_dir / "01_motor_no_load_phase_AB_detailed.png"
fig.savefig(phase_save_path, dpi=350)
print(f"Saved: {phase_save_path}")
plt.show()


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(18, 15), dpi=220)

# 1) Full time: |I1|
axes[0].plot(t, I1, color="#2ca02c", linewidth=1.1, label="|I1|")
axes[0].axvspan(0.0, startup_end, color="#fde2e4", alpha=0.35, label="startup window")
axes[0].axvspan(t[-1] - steady_window, t[-1], color="#e2f0cb", alpha=0.35, label="steady window")
axes[0].axhline(I1_mean_steady, color="#006d2c", linestyle="--", linewidth=1.2, label=f"steady mean = {I1_mean_steady:.1f} A")
axes[0].set_xlim(t[0], t[-1])
axes[0].set_ylim(*y_limits(I1, pad=0.08))
style_axis(axes[0], "Resultant Current Magnitude |I1| - Full Interval", xbins=24, ybins=16, x_minor=5, y_minor=5)
mark_point_with_projection(axes[0], t_I1_peak_start, I1_peak_start, "Startup peak |I1|", "#2ca02c", (10, 10))
mark_point_with_projection(axes[0], t_I1_mean_steady, I1_mean_steady, "Steady mean |I1|", "#006d2c", (10, -28))
axes[0].legend(loc="upper right")

# 2) Startup zoom: |I1|
axes[1].plot(t_start, I1_start, color="#2ca02c", linewidth=1.2, label="|I1|")
axes[1].set_xlim(0.0, startup_end)
axes[1].set_ylim(*y_limits(I1_start, pad=0.10))
style_axis(axes[1], "Resultant Current Magnitude |I1| - Startup Zoom", xbins=22, ybins=16, x_minor=5, y_minor=5)
mark_point_with_projection(axes[1], t_I1_peak_start, I1_peak_start, "Startup peak |I1|", "#2ca02c", (10, 10))
axes[1].legend(loc="upper right")

# 3) Steady-state zoom: |I1|
axes[2].plot(t_steady_zoom, I1_steady_zoom, color="#2ca02c", linewidth=1.2, label="|I1|")
axes[2].axhline(I1_mean_steady, color="#006d2c", linestyle="--", linewidth=1.2, label=f"steady mean = {I1_mean_steady:.1f} A")
axes[2].set_xlim(t_steady_zoom[0], t_steady_zoom[-1])
axes[2].set_ylim(*y_limits(I1_steady_zoom, pad=0.10))
style_axis(axes[2], "Resultant Current Magnitude |I1| - Steady-State Zoom", xbins=18, ybins=16, x_minor=5, y_minor=5)
mark_point_with_projection(axes[2], t_I1_peak_steady, I1_peak_steady, "Steady peak |I1|", "#2ca02c", (10, 10))
mark_point_with_projection(axes[2], t_I1_mean_steady, I1_mean_steady, "Steady mean |I1|", "#006d2c", (10, -28))
axes[2].legend(loc="upper right")

fig.suptitle("Motor No-Load: Detailed Resultant Current |I1| (High Resolution)", fontsize=15, weight="bold")
fig.subplots_adjust(left=0.10, right=0.98, top=0.94, bottom=0.06, hspace=0.36)
mod_save_path = output_dir / "01_motor_no_load_I1_mod_detailed.png"
fig.savefig(mod_save_path, dpi=350)
print(f"Saved: {mod_save_path}")
plt.show()
